<a href="https://colab.research.google.com/github/rishadb/sample_codes/blob/main/3CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CNN

import torch, torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#HP
learning_rate = 0.001
batch_size = 64
num_epochs = 2
load_state = True
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5,.5,.5), (.5,.5,.5))])

#load data
train_dataset = torchvision.datasets.CIFAR10(download = True, root = "//content/drive/MyDrive", train = True, transform = transform )
test_dataset = torchvision.datasets.CIFAR10(transform = transform, root = "/content/drive/MyDrive", train = False, download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True, num_workers= 2)
test_loader = DataLoader(dataset = test_dataset, shuffle = True, batch_size = batch_size, num_workers = 2)
train_dataset.classes #gives the classes in the dataset
#custom Data  
#data_dir = "dataDirHere"
#img_datasets = {x: torchvision.dataset.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
#dataloaders = {x: DataLoader(dataset = img_datasets[x], shuffle = True, batch_size=batch_size) for x in ['train', 'val']}

#defines:
class NN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)# in_channels, out_channel, kernel size
    self.pool1 = nn.MaxPool2d(2, 2)#kerrnel size, stride
    self.conv2 = nn.Conv2d(6, 16, 5)# n_out = ((n_in+ 2p - k)/s) + 1
    self.fc1 = nn.Linear(16*5*5, 120)#ouptput size of prev layer is 16-channel size,5-width,5-height as per the eqn
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    x = self.pool1(F.relu(self.conv1(x)))
    x = self.pool1(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

#to save/ load checkpoint
# def save_checkpoint(checkpoint, file = '/content/drive/MyDrive/Colab Notebooks/checkpoints/mycheckpoint.pth.tar'):
#   print("=> saving checkpoint")
#   torch.save(checkpoint, file)

# def load_checkpoint(location):
#   print("=> loading checkpoint")
#   model.load_state_dict(checkpoint["model_dict"])
#   optimizer.load_state_dict(checkpoint["optim_dict"])

  # defenition:
model = NN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
if load_state:
  #loading saved params from file
  loaded_states = torch.load("/content/drive/MyDrive/Colab Notebooks/checkpoints/mycheckpoint.pth.tar")
  model.load_state_dict(loaded_states["model_dict"])
  optimizer.load_state_dict(loaded_states["optim_dict"])

#train

for epoch in range(num_epochs):

#Saving the params to file
  if epoch == num_epochs-1:
    checkpoint = {'model_dict': model.state_dict(), 'optim_dict': optimizer.state_dict()}
    torch.save(checkpoint, "/content/drive/MyDrive/Colab Notebooks/checkpoints/mycheckpoint.pth.tar") # predefined func to save checkpoint

#save cpu, load cpu is the given case;
#save gpu load cpu: torch.load(path, map_location = device_cpu) 
#both on gpu: model.to(device_cuda) after loading
#save cpu load gpu: map_location = "cuda:0", model.to(device_cuda) after loading only

  for data, targets in train_loader:
    data = data.to(device)
    targets = targets.to(device) 
    score = model(data)
    loss = criterion(score, targets)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    


def acc(model, loader):
  model.eval()
  with torch.no_grad():
    classes = ["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    n_samples = 0
    n_correct = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in loader:
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)

      _, predictions = outputs.max(1)
      n_samples += labels.size(0)
      n_correct += (predictions == labels).sum().item()
      for i in range(10):
        label = labels[i]
        pred = predictions[i]
        if label ==pred:
          n_class_correct[label] +=1
        n_class_samples[label] +=1

  acc = n_correct/ n_samples
  print(f"total accuracy is {acc * 100}%")
  class_acc = [n_class_correct[i]/ n_class_samples[i] for i in range(10)]

  for i in range(10):
    print(f"{classes[i]} accuracy: {class_acc[i] * 100}%")

  return acc


acc(model, train_loader)
acc(model, test_loader)

cpu
Files already downloaded and verified
Files already downloaded and verified
total accuracy is 61.873999999999995%
plane accuracy: 65.15544041450777%
car accuracy: 80.9651474530831%
bird accuracy: 49.32182490752158%
cat accuracy: 47.20101781170484%
deer accuracy: 47.58713136729222%
dog accuracy: 59.00383141762452%
frog accuracy: 66.19537275064268%
horse accuracy: 64.07263294422827%
ship accuracy: 72.10460772104608%
truck accuracy: 65.29126213592234%
total accuracy is 59.38%
plane accuracy: 67.29559748427673%
car accuracy: 82.43243243243244%
bird accuracy: 49.67741935483871%
cat accuracy: 48.38709677419355%
deer accuracy: 36.96969696969697%
dog accuracy: 55.4140127388535%
frog accuracy: 61.78343949044586%
horse accuracy: 55.70469798657718%
ship accuracy: 72.8395061728395%
truck accuracy: 61.963190184049076%


0.5938

In [ ]:
#Transfer Learning
  #import pretrained model
#when making custom data, train, test folder, then folders with label name
import torch, torchvision
import time
import copy
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#HP
learning_rate = 0.001
batch_size = 64
num_epochs = 2
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5,.5,.5), (.5,.5,.5))])

#load data
train_dataset = torchvision.datasets.CIFAR10(download = True, root = "//content/drive/MyDrive", train = True, transform = transform )
test_dataset = torchvision.datasets.CIFAR10(transform = transform, root = "/content/drive/MyDrive", train = False, download = True)
image_datasets = {'train': train_dataset, 'val': test_dataset}
#train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True, num_workers= 2)
dataloaders = {x: DataLoader(dataset = train_dataset if x=='train' else test_dataset, batch_size=batch_size, shuffle=True, num_workers= 2)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)


class Identity(nn.Module): # takes in and returns same
  def __init__(self):
    super().__init__()
  def forward(self, x):
    return x

#loading the model
model = torchvision.models.vgg16(pretrained=True)

for param in model.parameters():
  param.requires_grad = False # freeze learning on pretrained params

#print(model) #gives all layers in the model
model.avgpool = Identity() #whole avgpool in the model replaced by identity
model.classifier= nn.Linear(512, 10) #1st classifier layer turned to linear layer defined
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)


def train_model(model, criterion, optimizer, scheduler = None, num_epochs=2):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train' and scheduler:
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

  
model_t = train_model(model, criterion, optimizer)

      

cuda
Files already downloaded and verified
Files already downloaded and verified
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Epoch 0/1
----------
train Loss: 1.1504 Acc: 0.6054
val Loss: 1.0361 Acc: 0.6382

Epoch 1/1
----------
train Loss: 0.9853 Acc: 0.6588
val Loss: 1.0062 Acc: 0.6505

Training complete in 0m 30s
Best val Acc: 0.650500
